In [1]:
import requests
import pandas as pd
from pathlib import Path
import time

BASE_URL = "https://collectionapi.metmuseum.org/public/collection/v1"
IMG_DIR = Path("images")
IMG_DIR.mkdir(exist_ok=True)

In [2]:
# Get object IDs with images
search_params = {"hasImages": True, "q": "*"}
object_ids = requests.get(f"{BASE_URL}/search", params=search_params).json()["objectIDs"]

records = []

for i, object_id in enumerate(object_ids[:500]):  # start small
    obj = requests.get(f"{BASE_URL}/objects/{object_id}").json()

    img_url = obj.get("primaryImage")
    if not img_url:
        continue

    # Download image
    img_path = IMG_DIR / f"{object_id}.jpg"
    if not img_path.exists():
        img_data = requests.get(img_url).content
        img_path.write_bytes(img_data)

    records.append({
        "objectID": object_id,
        "title": obj.get("title"),
        "artist": obj.get("artistDisplayName"),
        "department": obj.get("department"),
        "objectDate": obj.get("objectDate"),
        "medium": obj.get("medium"),
        "image_path": str(img_path)
    })

    if i % 50 == 0:
        time.sleep(0.2)

pd.DataFrame(records).to_csv("met_data.csv", index=False)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)